# lpcnet
[![Generic badge](https://img.shields.io/badge/GitHub-packname-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]

Author: [tarepan]

[github]:https://github.com/tarepan/LPCNetTorch
[notebook]:https://colab.research.google.com/github/tarepan/LPCNetTorch/blob/main/lpcnet.ipynb
[tarepan]:https://github.com/tarepan

## Colab Check
Check environments.

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}' # Google Colaboratory runnning time
!head -n 1 /proc/driver/nvidia/gpus/**/information                  # GPU type
!/usr/local/cuda/bin/nvcc --version | sed '4!d'                     # CUDA version
!python --version                                                   # Python version
!pip show torch | sed '2!d'                                         # PyTorch version

## Setup

Install tarepan/LPCNet for preprocessing

In [ ]:
# Step 0 - Install
!apt install autoconf automake libtool
!pip install git+https://github.com/tarepan/speechcorpusy.git
!git clone https://github.com/tarepan/LPCNet.git
%cd LPCNet
!./download_model.sh

# Step 1 - Env
%env CFLAGS=-Ofast -g -march=native
!echo $CFLAGS

# Step 2 - Build
!./autogen.sh    # Latest model download & `autoreconf`
!./configure     # Run the generated configure script
!make

Install the package from `tarepan/LPCNetTorch` public repository

In [ ]:
# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

# Dedicated dependencies install
# !pip install "torch==1.12.0" -q      # Based on your PyTorch environment
# !pip install "torchaudio==0.12.0" -q # Based on your PyTorch environment

# repository install
!pip uninstall lpcnet -y -q
!pip install git+https://github.com/tarepan/LPCNetTorch -q

## Preprocessing

In [ ]:
import librosa
import numpy as np
import soundfile as sf
import resampy
from speechcorpusy import load_preset


corpus = load_preset("Act100TKYM", root="/content/gdrive/MyDrive/ML_data")
corpus.get_contents()
all_utterances = corpus.get_identities()


path_outfile = "./train_pcm.s16"
sr_target = 16000

with open(path_outfile, mode="ab") as f:
  for p in map(lambda item_id: corpus.get_item_path(item_id), all_utterances):

    wave_int16, sr_source = sf.read(p, dtype='int16')
    # todo: force mono
    wave_int16_16k = resampy.resample(wave_int16, sr_source, sr_target)
    # Append headless 16-bit PCM
    wave_int16_16k.tofile(f)

In [ ]:
!./dump_data -train train_pcm.s16 train_features.f32 train_waves.s16

## Training

In [ ]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/gdrive/MyDrive/ML_results/lpcnet/torch

# Train
!python -m lpcnet.main_train \
    train.ckpt_log.dir_root=/content/gdrive/MyDrive/ML_results/lpcnet/torch \
    train.ckpt_log.name_exp=2022 \
    train.ckpt_log.name_version=version_1 \
    data.adress_data_root=/content/gdrive/MyDrive/ML_data \


## Inference

### From CLI

In [ ]:
!python -m lpcnet.main_inference \
    --model-ckpt-path="gdrive/MyDrive/ML_results/lpcnet/test2/default/version_0/checkpoints/last.ckpt" \
    --i-path="./test.wav" \
    --o-path="./o.wav" \
    # --device="cpu" --device="cuda:0" \

### From Python

In [ ]:
import torch

from lpcnet.model import Model


# Configs
model_ckpt_path = "<ckpt.pt>"
device = "cuda:0" # "cpu"

# Setup
model: Model = Model.load_from_checkpoint(checkpoint_path=model_ckpt_path).to(torch.device(device)) # type: ignore ; because of PyTorch Lightning
model.eval()

# Inference
with torch.inference_mode():
    # Raw data
    ####################################
    ## From sample
    raw = model.sample()
    ####################################
    ## From your file
    # from pathlib import Path
    # i_path = Path(f"<your_data>.xxx")
    # raw = model.load(i_path)
    ####################################
    ## From your upstream data
    # raw = <your_raw_data>
    ####################################

    batch = model.preprocess(raw, device)
    o_pred = model.predict_step(batch, batch_idx=0)

    # Tensor[Batch=1, ...] => Tensor[...] => NDArray[...]
    o_wave = o_pred[0].to('cpu').numpy()

# Output
print(o_wave)
##################################################
# Audio
##############################################
## To File
# import soundfile as sf
# sf.write(...)
##############################################
## To Notebook
# from IPython.display import Audio, display
# display(Audio(o_wave, rate=o_sr))
##############################################
##################################################


In [ ]:
# # Usage stat
# ## GPU
# !nvidia-smi -l 3
# ## CPU
# !vmstat 5
# !top